In [61]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery


In [62]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\RAIS')


In [63]:
os.listdir()

['RAIS_poderes_esferas.xlsx',
 'RAIS_poderes_esferas.csv',
 'RAIS_remuneracao_poder_efera.xlsx',
 'RAIS_remuneracao_poder_efera.csv',
 'indicador_salarios_rais2022.xlsx']

In [64]:
df = pd.read_excel('indicador_salarios_rais2022.xlsx', usecols="A:C")
df

,Medida,Valor,categoria
0,1º Decil,1300.000,Legislativo Estadual
1,1º Decil,1300.876,Legislativo Municipal
2,1º Decil,1313.000,Municipal
3,1º Decil,1313.000,Executivo Municipal
4,1º Decil,1328.410,Legislativo
...,...,...,...
160,9º Decil,25332.020,Legislativo Estadual
161,9º Decil,25971.110,Judiciário
162,9º Decil,26430.576,Judiciário Federal
163,8º Decil,31428.930,Legislativo Federal


In [65]:
df = df[df['Medida']=='Mediana']

In [66]:
def agrupamento (df):
    if df['categoria'] in ['Executivo Municipal', 'Legislativo Municipal', 'Executivo Estadual', 'Legislativo Estadual', 'Executivo Federal', 'Legislativo Federal', 'Judiciário Estadual', 'Judiciário Federal']:
        return 'Esfera e Poder'
    elif df['categoria'] in ['Municipal', 'Federal', 'Estadual', 'Público']:
        return 'Esfera'
    elif df['categoria'] in ['Executivo', 'Legislativo', 'Judiciário']:
        return 'Poder'


In [67]:
df['grupo'] = df.apply(agrupamento, axis=1) #incluindo a função acima na coluna denominada "raça"

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_15272\2225536961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grupo'] = df.apply(agrupamento, axis=1) #incluindo a função acima na coluna denominada "raça"


In [68]:
df['grupo_ordenado'] = pd.factorize(df['grupo'])[0]  
df

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_15272\240387055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grupo_ordenado'] = pd.factorize(df['grupo'])[0]


,Medida,Valor,categoria,grupo,grupo_ordenado
40,Mediana,2795.780,Executivo Municipal,Esfera e Poder,0
41,Mediana,2800.330,Municipal,Esfera,1
46,Mediana,2967.010,Legislativo Municipal,Esfera e Poder,0
52,Mediana,3386.200,Executivo,Poder,2
54,Mediana,3567.290,Público,Esfera,1
59,Mediana,3882.760,Legislativo,Poder,2
68,Mediana,4525.540,Executivo Estadual,Esfera e Poder,0
75,Mediana,4848.100,Estadual,Esfera,1
90,Mediana,6374.525,Legislativo Estadual,Esfera e Poder,0
105,Mediana,7865.930,Executivo Federal,Esfera e Poder,0


In [69]:
df_ordenado = df.sort_values(by='grupo_ordenado').drop(columns=['grupo_ordenado', 'Medida']).reset_index(drop=True)


In [70]:
df_ordenado = df_ordenado.rename(columns={'Valor':'mediana', 'categoria':'poder_esfera'})
df_ordenado

,mediana,poder_esfera,grupo
0,2795.780,Executivo Municipal,Esfera e Poder
1,2967.010,Legislativo Municipal,Esfera e Poder
2,4525.540,Executivo Estadual,Esfera e Poder
3,6374.525,Legislativo Estadual,Esfera e Poder
4,7865.930,Executivo Federal,Esfera e Poder
5,7875.850,Legislativo Federal,Esfera e Poder
6,10071.080,Judiciário Estadual,Esfera e Poder
7,15463.420,Judiciário Federal,Esfera e Poder
8,2800.330,Municipal,Esfera
9,3567.290,Público,Esfera


Subindo para o GBQ

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [ ]:
df_ordenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mediana       15 non-null     float64
 1   poder_esfera  15 non-null     object 
 2   grupo         15 non-null     object 
dtypes: float64(1), object(2)
memory usage: 492.0+ bytes


In [ ]:
schema = [bigquery.SchemaField('poder_esfera', 'STRING', description= 'Nível da esfera e poder de governo'),
          bigquery.SchemaField('grupo', 'STRING', description= 'Se é esfera, poder ou esfera e poder'),
          bigquery.SchemaField('mediana', 'FLOAT', description= 'Mediana da remuneração por grupo')]

In [ ]:
table_ref = dataset_ref.table('RAIS_remuneracao_poder_esfera')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df_ordenado, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=0430aa64-3abe-487c-b7c9-70684d493de1>

In [ ]:
df_ordenado

,mediana,poder_esfera,grupo
0,2795.780,Executivo Municipal,Esfera e Poder
1,2967.010,Legislativo Municipal,Esfera e Poder
2,4525.540,Executivo Estadual,Esfera e Poder
3,6374.525,Legislativo Estadual,Esfera e Poder
4,7865.930,Executivo Federal,Esfera e Poder
5,7875.850,Legislativo Federal,Esfera e Poder
6,10071.080,Judiciário Estadual,Esfera e Poder
7,15463.420,Judiciário Federal,Esfera e Poder
8,2800.330,Municipal,Esfera
9,3567.290,Público,Esfera
